# Imports

In [ ]:
from numpy.random import seed
seed(888)
from tensorflow import random
random.set_seed(404)

In [ ]:
import os
import numpy as np
import tensorflow as tf

from time import strftime
from PIL import Image

# Constants

In [ ]:
X_TRAIN_PATH = 'MNIST/digit_xtrain.csv'
X_TEST_PATH = 'MNIST/digit_xtest.csv'
Y_TRAIN_PATH = 'MNIST/digit_ytrain.csv'
Y_TEST_PATH = 'MNIST/digit_ytest.csv'

LOGGING_PATH = 'tensorboard_mnist_digit_logs/'

NR_CLASSES = 10
VALIDATION_SIZE = 10000
IMAGE_WIDTH = 28
IMAGE_HEIGHT = 28
CHANNELS = 1
TOTAL_INPUTS = IMAGE_WIDTH*IMAGE_HEIGHT*CHANNELS

# Get the Data

In [ ]:
%%time

y_train_all = np.loadtxt(Y_TRAIN_PATH, delimiter=',', dtype=int)

In [ ]:
y_train_all.shape

In [ ]:
y_test = np.loadtxt(Y_TEST_PATH, delimiter=',', dtype=int)

In [ ]:
%%time

x_train_all = np.loadtxt(X_TRAIN_PATH, delimiter=',', dtype=int)

In [ ]:
%%time

x_test = np.loadtxt(X_TEST_PATH, delimiter=',', dtype=int)

# Explore

In [ ]:
x_train_all.shape

In [ ]:
x_train_all[0]

In [ ]:
y_train_all.shape

In [ ]:
x_test.shape

In [ ]:
y_train_all[:5]

# Data Preprocessing

In [ ]:
# Re-scale
x_train_all, x_test = x_train_all / 255.0, x_test / 255.0

### Convert target values to one-hot encoding

In [ ]:
values = y_train_all[:5]
np.eye(10)[values]

In [ ]:
np.eye(10)

In [ ]:
np.eye(10)[2]

In [ ]:
values

In [ ]:
values[4]

In [ ]:
y_train_all = np.eye(NR_CLASSES)[y_train_all]

In [ ]:
y_train_all.shape

In [ ]:
y_test = np.eye(NR_CLASSES)[y_test]
y_test.shape

## Create validation dataset from training data

In [ ]:
x_val = x_train_all[:VALIDATION_SIZE]
y_val = y_train_all[:VALIDATION_SIZE]

In [ ]:
x_train = x_train_all[VALIDATION_SIZE:]
y_train = y_train_all[VALIDATION_SIZE:]

In [ ]:
x_train.shape

In [ ]:
x_val.shape

In [ ]:
class LogImages(tf.keras.callbacks.Callback):
    def __init__(self, log_dir, x_data):
        super(LogImages, self).__init__()
        self.log_dir = log_dir
        self.x_data = x_data 
        
    def on_epoch_end(self, epoch, logs=None):
        # Log a batch of images at the end of an epoch
        file_writer = tf.summary.create_file_writer(self.log_dir + '/images')

        with file_writer.as_default():
            images = np.reshape(self.x_data[:4], (-1, 28, 28, 1))  # Log 4 images
            tf.summary.image("4 training data examples", images, max_outputs=4, step=epoch)

# Load the training Image

In [ ]:
from tensorflow.keras.preprocessing.image import img_to_array, load_img

img_path = '/Users/kenny/Documents/DS Projects/MathGarden/MNIST/test_img.png'

img = load_img(img_path, color_mode='grayscale', target_size=(28, 28))

img_array = img_to_array(img) / 255.0

test_img = img_array.flatten()


# Setup Tensorflow Graph

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout

# Neural network architecture parameters
n_hidden1 = 512  
n_hidden2 = 64   
NR_CLASSES = 10  
nr_epochs = 10   

# Define the Model
model = Sequential([
    Dense(n_hidden1, activation='relu', input_shape=(TOTAL_INPUTS,), name='layer_1'),  # Note the input_shape adjustment
    Dropout(0.2), 
    Dense(n_hidden2, activation='relu', name='layer_2'),
    Dense(NR_CLASSES, activation='softmax', name='output_layer')
])


# Compile the Model
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

# Display model's architecture
model.summary()

# Train the Model
history = model.fit(x_train, y_train, epochs=nr_epochs, batch_size=1000, validation_data=(x_val, y_val))

# Evaluate the Model on the test data
test_loss, test_accuracy = model.evaluate(x_test, y_test)
print(f"Test loss: {test_loss}, Test accuracy: {test_accuracy}")

# Model Predictions
predictions = model.predict(np.array([test_img])) 
predicted_class = np.argmax(predictions, axis=1)
print(f"Prediction for test image is {predicted_class}")


## Neural Network Architecture

In [ ]:
nr_epochs = 50
learning_rate = 1e-3

n_hidden1 = 512
n_hidden2 = 64

In [ ]:
model = Sequential([
    Dense(n_hidden1, activation='relu', input_shape=(TOTAL_INPUTS,), name='layer_1'),
    Dropout(0.2),
    Dense(n_hidden2, activation='relu', name='layer_2'),
    Dense(NR_CLASSES, activation='softmax', name='output_layer')
])

model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])


# Tensorboard Setup

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir logs/fit


# Loss, Optimisation & Metrics

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# Constants for the model
n_hidden1 = 512  
n_hidden2 = 64   
NR_CLASSES = 10  
TOTAL_INPUTS = 784 

# Define the Model
model = Sequential([
    Dense(n_hidden1, activation='relu', input_shape=(TOTAL_INPUTS,), name='layer_1'),
    Dropout(0.2), 
    Dense(n_hidden2, activation='relu', name='layer_2'),
    Dense(NR_CLASSES, activation='softmax', name='output_layer')
])

# Compile the Model
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

# Display model's architecture
model.summary()


# Run Session

In [ ]:
# Define the batch size for training
size_of_batch = 1000 

history = model.fit(x_train, y_train,
                    epochs=nr_epochs,
                    batch_size=size_of_batch,
                    validation_data=(x_val, y_val))

print(history.history)

# Evaluate the Model on the test dataset
test_loss, test_accuracy = model.evaluate(x_test, y_test)
print(f'Test Loss: {test_loss}, Test Accuracy: {test_accuracy}')


# Make a Prediction

In [ ]:
from tensorflow.keras.preprocessing.image import img_to_array, load_img

# Load the image, ensuring it is in grayscale, and resize it to 28x28 pixels
img = load_img('MNIST/test_img.png', color_mode='grayscale', target_size=(28, 28))

# Convert the image to an array, normalize it, and possibly invert if required
img_array = img_to_array(img) / 255.0

# If your model was trained on inverted images, invert the pixel values
# img_array = 1.0 - img_array

# Flatten the array if your model expects flattened input
test_img = img_array.flatten()

# Make the prediction
predictions = model.predict(np.array([test_img]))
predicted_class = np.argmax(predictions, axis=1)
print(f'Prediction for test image is {predicted_class}')


In [ ]:
# Evaluate the model on the test dataset
test_loss, test_accuracy = model.evaluate(x_test, y_test)
print(f'Accuracy on test set is {test_accuracy:.2%}')


# Imports

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.callbacks import TensorBoard
from PIL import Image
from datetime import datetime
import os

# Seed for reproducing

In [ ]:
np.random.seed(888)
tf.random.set_seed(404)

# Constants

In [ ]:
X_TRAIN_PATH = 'MNIST/digit_xtrain.csv'
X_TEST_PATH = 'MNIST/digit_xtest.csv'
Y_TRAIN_PATH = 'MNIST/digit_ytrain.csv'
Y_TEST_PATH = 'MNIST/digit_ytest.csv'
NR_CLASSES = 10
VALIDATION_SIZE = 10000
IMAGE_WIDTH = 28
IMAGE_HEIGHT = 28
CHANNELS = 1
TOTAL_INPUTS = IMAGE_WIDTH * IMAGE_HEIGHT * CHANNELS

# Get the Data

In [ ]:
y_train_all = np.loadtxt(Y_TRAIN_PATH, delimiter=',', dtype=int)
x_train_all = np.loadtxt(X_TRAIN_PATH, delimiter=',', dtype=int)
x_test = np.loadtxt(X_TEST_PATH, delimiter=',', dtype=int)
y_test = np.loadtxt(Y_TEST_PATH, delimiter=',', dtype=int)


# Data Preprocessing

In [ ]:
x_train_all, x_test = x_train_all / 255.0, x_test / 255.0
y_train_all = np.eye(NR_CLASSES)[y_train_all]
y_test = np.eye(NR_CLASSES)[y_test]

# Create validation dataset from training data

In [ ]:
x_val = x_train_all[:VALIDATION_SIZE]
y_val = y_train_all[:VALIDATION_SIZE]
x_train = x_train_all[VALIDATION_SIZE:]
y_train = y_train_all[VALIDATION_SIZE:]

# Neural Network Architecture

In [ ]:
nr_epochs = 50
learning_rate = 1e-3
n_hidden1 = 512
n_hidden2 = 64

# Define the Model

In [ ]:
model = Sequential([
    Flatten(input_shape=(TOTAL_INPUTS,)),
    Dense(n_hidden1, activation='relu'),
    Dropout(0.8),
    Dense(n_hidden2, activation='relu'),
    Dense(NR_CLASSES, activation='softmax')
])

# Compile the Model

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# TensorBoard Setup

In [ ]:
log_dir = os.path.join("logs", "fit", datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

# Training the Model with TensorBoard

In [ ]:
history = model.fit(
    x_train, y_train, 
    epochs=nr_epochs, 
    batch_size=1000, 
    validation_data=(x_val, y_val), 
    callbacks=[tensorboard_callback]
)

# Making a Prediction

In [ ]:
def make_prediction(image_path):
    img = Image.open(image_path)
    bw = img.convert('L')
    img_array = np.invert(bw)
    test_img = img_array.ravel() / 255.0
    prediction = model.predict(np.array([test_img]))
    return np.argmax(prediction)

# Example of making a prediction

In [ ]:
prediction = make_prediction('MNIST/test_img.png')
print(f'Prediction for test image is {prediction}')

# Testing and Evaluation

In [ ]:
test_loss, test_accuracy = model.evaluate(x_test, y_test)
print(f"Test loss: {test_loss}, Test accuracy: {test_accuracy:.2%}")

In [ ]:
# Imports

import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.callbacks import TensorBoard
from PIL import Image
from datetime import datetime
import os

# Seed for reproducing

np.random.seed(888)
tf.random.set_seed(404)

# Constants

X_TRAIN_PATH = 'MNIST/digit_xtrain.csv'
X_TEST_PATH = 'MNIST/digit_xtest.csv'
Y_TRAIN_PATH = 'MNIST/digit_ytrain.csv'
Y_TEST_PATH = 'MNIST/digit_ytest.csv'
NR_CLASSES = 10
VALIDATION_SIZE = 10000
IMAGE_WIDTH = 28
IMAGE_HEIGHT = 28
CHANNELS = 1
TOTAL_INPUTS = IMAGE_WIDTH * IMAGE_HEIGHT * CHANNELS

# Get the Data

y_train_all = np.loadtxt(Y_TRAIN_PATH, delimiter=',', dtype=int)
x_train_all = np.loadtxt(X_TRAIN_PATH, delimiter=',', dtype=int)
x_test = np.loadtxt(X_TEST_PATH, delimiter=',', dtype=int)
y_test = np.loadtxt(Y_TEST_PATH, delimiter=',', dtype=int)


# Data Preprocessing

x_train_all, x_test = x_train_all / 255.0, x_test / 255.0
y_train_all = np.eye(NR_CLASSES)[y_train_all]
y_test = np.eye(NR_CLASSES)[y_test]

# Create validation dataset from training data

x_val = x_train_all[:VALIDATION_SIZE]
y_val = y_train_all[:VALIDATION_SIZE]
x_train = x_train_all[VALIDATION_SIZE:]
y_train = y_train_all[VALIDATION_SIZE:]

# Neural Network Architecture

nr_epochs = 50
learning_rate = 1e-3
n_hidden1 = 512
n_hidden2 = 64

# Define the Model

model = Sequential([
    Flatten(input_shape=(TOTAL_INPUTS,)),
    Dense(n_hidden1, activation='relu'),
    Dropout(0.8),
    Dense(n_hidden2, activation='relu'),
    Dense(NR_CLASSES, activation='softmax')
])

# Compile the Model

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# TensorBoard Setup

log_dir = os.path.join("logs", "fit", datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

# Training the Model with TensorBoard

history = model.fit(
    x_train, y_train, 
    epochs=nr_epochs, 
    batch_size=1000, 
    validation_data=(x_val, y_val), 
    callbacks=[tensorboard_callback]
)

# Making a Prediction

def make_prediction(image_path):
    img = Image.open(image_path)
    bw = img.convert('L')
    img_array = np.invert(bw)
    test_img = img_array.ravel() / 255.0
    prediction = model.predict(np.array([test_img]))
    return np.argmax(prediction)

# Example of making a prediction

prediction = make_prediction('MNIST/test_img.png')
print(f'Prediction for test image is {prediction}')

# Testing and Evaluation

test_loss, test_accuracy = model.evaluate(x_test, y_test)
print(f"Test loss: {test_loss}, Test accuracy: {test_accuracy:.2%}")

# Imports

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.callbacks import TensorBoard
from PIL import Image
from datetime import datetime
import os

# Seed for reproducing

In [ ]:
np.random.seed(888)
tf.random.set_seed(404)

# Constants

In [ ]:
X_TRAIN_PATH = 'MNIST/digit_xtrain.csv'
X_TEST_PATH = 'MNIST/digit_xtest.csv'
Y_TRAIN_PATH = 'MNIST/digit_ytrain.csv'
Y_TEST_PATH = 'MNIST/digit_ytest.csv'
NR_CLASSES = 10
VALIDATION_SIZE = 10000
IMAGE_WIDTH = 28
IMAGE_HEIGHT = 28
CHANNELS = 1
TOTAL_INPUTS = IMAGE_WIDTH * IMAGE_HEIGHT * CHANNELS

# Get the Data

In [ ]:
y_train_all = np.loadtxt(Y_TRAIN_PATH, delimiter=',', dtype=int)
x_train_all = np.loadtxt(X_TRAIN_PATH, delimiter=',', dtype=int)
x_test = np.loadtxt(X_TEST_PATH, delimiter=',', dtype=int)
y_test = np.loadtxt(Y_TEST_PATH, delimiter=',', dtype=int)


# Data Preprocessing

In [ ]:
x_train_all, x_test = x_train_all / 255.0, x_test / 255.0
y_train_all = np.eye(NR_CLASSES)[y_train_all]
y_test = np.eye(NR_CLASSES)[y_test]

# Create validation dataset from training data

In [ ]:
x_val = x_train_all[:VALIDATION_SIZE]
y_val = y_train_all[:VALIDATION_SIZE]
x_train = x_train_all[VALIDATION_SIZE:]
y_train = y_train_all[VALIDATION_SIZE:]

# Neural Network Architecture

In [ ]:
nr_epochs = 50
learning_rate = 1e-3
n_hidden1 = 512
n_hidden2 = 64

# Define the Model

In [ ]:
model = Sequential([
    Flatten(input_shape=(TOTAL_INPUTS,)),
    Dense(n_hidden1, activation='relu'),
    Dropout(0.8),
    Dense(n_hidden2, activation='relu'),
    Dense(NR_CLASSES, activation='softmax')
])

# Compile the Model

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# TensorBoard Setup

In [ ]:
log_dir = os.path.join("logs", "fit", datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

# Training the Model with TensorBoard

In [ ]:
history = model.fit(
    x_train, y_train, 
    epochs=nr_epochs, 
    batch_size=1000, 
    validation_data=(x_val, y_val), 
    callbacks=[tensorboard_callback]
)

# Making a Prediction

In [ ]:
def make_prediction(image_path):
    img = Image.open(image_path)
    bw = img.convert('L')
    img_array = np.invert(bw)
    test_img = img_array.ravel() / 255.0
    prediction = model.predict(np.array([test_img]))
    return np.argmax(prediction)

# Example of making a prediction

In [ ]:
prediction = make_prediction('MNIST/test_img.png')
print(f'Prediction for test image is {prediction}')

# Testing and Evaluation

In [ ]:
test_loss, test_accuracy = model.evaluate(x_test, y_test)
print(f"Test loss: {test_loss}, Test accuracy: {test_accuracy:.2%}")

# Imports

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model

# Constants

In [ ]:
X_TEST_PATH = 'MNIST/load_xtest.csv'
Y_TEST_PATH = 'MNIST/load_ytest.csv'

# Load the Data

In [ ]:
y_test = np.loadtxt(Y_TEST_PATH, delimiter=',', dtype=int)
x_test = np.loadtxt(X_TEST_PATH, delimiter=',', dtype=float)

# Constants

In [ ]:
NR_CLASSES = 10

# One-Hot Encode the Test Labels

In [ ]:
y_test_one_hot = np.eye(NR_CLASSES)[y_test]

# Load the Saved Model

In [ ]:
model = load_model('SavedModel/my_model')

# Make Prediction

In [ ]:
predictions = model.predict(x_test)
predicted_classes = np.argmax(predictions, axis=1)

# Check Results

In [ ]:
print(y_test[:5])

In [ ]:
print(predicted_classes[:5])

# Calculate Accuracy

In [ ]:
nr_correct = sum(np.equal(predicted_classes, y_test))
accuracy = nr_correct / len(predicted_classes)
print(f"Accuracy: {accuracy:.2%}")


# Evaluate the model on the test results

In [ ]:
test_loss, test_accuracy = model.evaluate(x_test, y_test_one_hot, verbose=2)
print(f"Test loss: {test_loss}, Test accuracy: {test_accuracy:.2%}")

# Imports

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.callbacks import TensorBoard
from PIL import Image
from datetime import datetime
import os

# Seed for reproducing

In [ ]:
np.random.seed(888)
tf.random.set_seed(404)

# Constants

In [ ]:
X_TRAIN_PATH = 'MNIST/digit_xtrain.csv'
X_TEST_PATH = 'MNIST/digit_xtest.csv'
Y_TRAIN_PATH = 'MNIST/digit_ytrain.csv'
Y_TEST_PATH = 'MNIST/digit_ytest.csv'
NR_CLASSES = 10
VALIDATION_SIZE = 10000
IMAGE_WIDTH = 28
IMAGE_HEIGHT = 28
CHANNELS = 1
TOTAL_INPUTS = IMAGE_WIDTH * IMAGE_HEIGHT * CHANNELS

# Get the Data

In [ ]:
y_train_all = np.loadtxt(Y_TRAIN_PATH, delimiter=',', dtype=int)
x_train_all = np.loadtxt(X_TRAIN_PATH, delimiter=',', dtype=int)
x_test = np.loadtxt(X_TEST_PATH, delimiter=',', dtype=int)
y_test = np.loadtxt(Y_TEST_PATH, delimiter=',', dtype=int)


# Data Preprocessing

In [ ]:
x_train_all, x_test = x_train_all / 255.0, x_test / 255.0
y_train_all = np.eye(NR_CLASSES)[y_train_all]
y_test = np.eye(NR_CLASSES)[y_test]

# Create validation dataset from training data

In [ ]:
x_val = x_train_all[:VALIDATION_SIZE]
y_val = y_train_all[:VALIDATION_SIZE]
x_train = x_train_all[VALIDATION_SIZE:]
y_train = y_train_all[VALIDATION_SIZE:]

# Neural Network Architecture

In [ ]:
nr_epochs = 50
learning_rate = 1e-3
n_hidden1 = 512
n_hidden2 = 64

# Define the Model

In [ ]:
model = Sequential([
    Flatten(input_shape=(TOTAL_INPUTS,)),
    Dense(n_hidden1, activation='relu'),
    Dropout(0.8),
    Dense(n_hidden2, activation='relu'),
    Dense(NR_CLASSES, activation='softmax')
])

# Compile the Model

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# TensorBoard Setup

In [ ]:
log_dir = os.path.join("logs", "fit", datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

# Training the Model with TensorBoard

In [ ]:
history = model.fit(
    x_train, y_train, 
    epochs=nr_epochs, 
    batch_size=1000, 
    validation_data=(x_val, y_val), 
    callbacks=[tensorboard_callback]
)

# Saving the Model

In [ ]:
model.save('SavedModel/my_model')

# Making a Prediction

In [ ]:
def make_prediction(image_path):
    img = Image.open(image_path)
    bw = img.convert('L')
    img_array = np.invert(bw)
    test_img = img_array.ravel() / 255.0
    prediction = model.predict(np.array([test_img]))
    return np.argmax(prediction)

# Example of making a prediction

In [ ]:
prediction = make_prediction('MNIST/test_img.png')
print(f'Prediction for test image is {prediction}')

# Testing and Evaluation

In [ ]:
test_loss, test_accuracy = model.evaluate(x_test, y_test)
print(f"Test loss: {test_loss}, Test accuracy: {test_accuracy:.2%}")